# 3D interpolation with time or 4D interpolation


In [2]:
import cartopy.crs
import matplotlib
import matplotlib.pyplot
import numpy
import pyinterp
import pyinterp.backends.xarray
import xarray
import warnings
warnings.filterwarnings('ignore')

In [ ]:
ds = xarray.open_dataset("pres_temp_4D.nc")
interpolator = pyinterp.backends.xarray.Grid4D(ds.pressure)

In [ ]:
mx, my, mz, mu = numpy.meshgrid(numpy.arange(-125, -70, 0.5),
                                numpy.arange(25, 50, 0.5),
                                numpy.datetime64("2000-01-01T12:00"),
                                0.5,
                                indexing="ij")

In [ ]:
quadrivariate = interpolator.quadrivariate(
    dict(longitude=mx.flatten(),
         latitude=my.flatten(),
         time=mz.flatten(),
         level=mu.flatten())).reshape(mx.shape)

In [ ]:
interpolator = pyinterp.backends.xarray.Grid4D(ds.pressure, increasing_axes=True)
bicubic = interpolator.bicubic(dict(longitude=mx.flatten(),
                                    latitude=my.flatten(),
                                    time=mz.flatten(),
                                    level=mu.flatten()),
                               nx=2,
                               ny=2).reshape(mx.shape)

In [ ]:
quadrivariate = quadrivariate.squeeze(axis=(2, 3))
bicubic = bicubic.squeeze(axis=(2, 3))
lons = mx[:, 0].squeeze()
lats = my[0, :].squeeze()

In [ ]:
fig = matplotlib.pyplot.figure(figsize=(5, 4))
ax1 = fig.add_subplot(211, projection=cartopy.crs.PlateCarree(central_longitude=180))
pcm = ax1.pcolormesh(lons,
                     lats,
                     quadrivariate.T,
                     cmap='jet',
                     transform=cartopy.crs.PlateCarree())
ax1.coastlines()
ax1.set_title("Trilinear")

ax2 = fig.add_subplot(212, projection=cartopy.crs.PlateCarree(central_longitude=180))
pcm = ax2.pcolormesh(lons,
                     lats,
                     bicubic.T,
                     cmap='jet',
                     transform=cartopy.crs.PlateCarree())
ax2.coastlines()
ax2.set_title("Spline & Linear in time")
fig.colorbar(pcm, ax=[ax1, ax2], shrink=0.8)
fig.show()

# Empirical Orthogonal Function (EOF) analysis 

In [ ]:
from pyEOF import *
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt